<a href="https://colab.research.google.com/github/liyaSileshi/amharic-sentiment-analysis/blob/main/tweet_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libraries

In [37]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
pd.options.mode.chained_assignment = None
import re

## Downloading the dataset

In [38]:
url = "https://raw.githubusercontent.com/liyaSileshi/amharic-sentiment-analysis/main/data_preprocess/test.csv"
df_test = pd.read_csv(url)
df_test

,Unnamed: 0,tweet_id,sentiment,tweet
0,0,1213842050455654402,neutral,@Haymana_ እንደዛ ካልሆነ በስተቀር ሰውዬው ማንንም አይቦልኩም
1,1,1215480766295171072,neutral,NaN
2,2,1222950264812064768,positive,ለመገናኛ ብዙሃን አባላት!\n\nበደምቢዶሎ የታገቱ ኢትዮጵያውያን ድምፅ ለ...
3,3,1213967203076395009,negative,NaN
4,4,1213961696840171521,neutral,NaN
...,...,...,...,...
934,934,1214922390469324802,neutral,@degu_zerihun የውስጥ ሰለማችን መጠበቅ፣ የልማታችን ሁሉ ወሳኝ ማ...
935,935,1214922390469324802,neutral,@degu_zerihun የውስጥ ሰለማችን መጠበቅ፣ የልማታችን ሁሉ ወሳኝ ማ...
936,936,1223053757883260928,positive,"""አዲሱ መንግስት""ና ህገ-መንግስታዊነት - ረዥም ገመድ\n\nህገ-መንግስታ..."
937,937,1213771045070659584,negative,@TeameYitbarek @Ze_Astegoma @eskinder_nega ትግሬ...


In [41]:
# remove null values from dataset
df_test = df_test.dropna()
df_test

,Unnamed: 0,tweet_id,sentiment,tweet
0,0,1213842050455654402,neutral,@Haymana_ እንደዛ ካልሆነ በስተቀር ሰውዬው ማንንም አይቦልኩም
2,2,1222950264812064768,positive,ለመገናኛ ብዙሃን አባላት!\n\nበደምቢዶሎ የታገቱ ኢትዮጵያውያን ድምፅ ለ...
5,5,1213760822964609024,positive,የምወድሽ፣\nየምሳሳልሽ፣\nእናቴ፣\nሃገሬ፣ \nኢትዮጵያ!\nከላይ አረንጓ...
6,6,1201134046862094336,positive,RT @AntyeaVerse: የዚህ ቤት ከፍለን የማንጨርሰው ውለታ አለብን ...
7,7,1214129980306808834,neutral,@IamWizEthiopia ልሙጦቹ ... ደግሞ ምን ይደረግላችሁ በበአሉ?
...,...,...,...,...
934,934,1214922390469324802,neutral,@degu_zerihun የውስጥ ሰለማችን መጠበቅ፣ የልማታችን ሁሉ ወሳኝ ማ...
935,935,1214922390469324802,neutral,@degu_zerihun የውስጥ ሰለማችን መጠበቅ፣ የልማታችን ሁሉ ወሳኝ ማ...
936,936,1223053757883260928,positive,"""አዲሱ መንግስት""ና ህገ-መንግስታዊነት - ረዥም ገመድ\n\nህገ-መንግስታ..."
937,937,1213771045070659584,negative,@TeameYitbarek @Ze_Astegoma @eskinder_nega ትግሬ...


In [42]:
#drop repeated rows (drop rows with similar tweet id)
df_test = df_test.drop_duplicates(subset='tweet_id', keep="first")
df_test

,Unnamed: 0,tweet_id,sentiment,tweet
0,0,1213842050455654402,neutral,@Haymana_ እንደዛ ካልሆነ በስተቀር ሰውዬው ማንንም አይቦልኩም
2,2,1222950264812064768,positive,ለመገናኛ ብዙሃን አባላት!\n\nበደምቢዶሎ የታገቱ ኢትዮጵያውያን ድምፅ ለ...
5,5,1213760822964609024,positive,የምወድሽ፣\nየምሳሳልሽ፣\nእናቴ፣\nሃገሬ፣ \nኢትዮጵያ!\nከላይ አረንጓ...
6,6,1201134046862094336,positive,RT @AntyeaVerse: የዚህ ቤት ከፍለን የማንጨርሰው ውለታ አለብን ...
7,7,1214129980306808834,neutral,@IamWizEthiopia ልሙጦቹ ... ደግሞ ምን ይደረግላችሁ በበአሉ?
...,...,...,...,...
931,931,1216256678808735744,neutral,• ከታች ያለው አማራ ክልል ነው! እዚያ አዲሥ ናቸው መሠለኝ ብሩን እያተ...
933,933,1214922390469324802,neutral,@degu_zerihun የውስጥ ሰለማችን መጠበቅ፣ የልማታችን ሁሉ ወሳኝ ማ...
936,936,1223053757883260928,positive,"""አዲሱ መንግስት""ና ህገ-መንግስታዊነት - ረዥም ገመድ\n\nህገ-መንግስታ..."
937,937,1213771045070659584,negative,@TeameYitbarek @Ze_Astegoma @eskinder_nega ትግሬ...


In [44]:
def clean_text(row, options):

    if options['lowercase']:
        row = row.lower()

    # if options['decode_html']:
    #     txt = BeautifulSoup(row, 'lxml')
    #     row = txt.get_text()

    if options['remove_url']:
        row = row.replace('http\S+|www.\S+', '')

    if options['remove_mentions']:
        row = re.sub("@[A-Za-z0-9]+","", row)

    return row

clean_config = {
    'remove_url': True,
    'remove_mentions': True,
    # 'decode_utf8': True,
    'lowercase': True
    }

df_test['tweet'] = df_test['tweet'].apply(clean_text, args=(clean_config,))

In [45]:
df_test

,Unnamed: 0,tweet_id,sentiment,tweet
0,0,1213842050455654402,neutral,_ እንደዛ ካልሆነ በስተቀር ሰውዬው ማንንም አይቦልኩም
2,2,1222950264812064768,positive,ለመገናኛ ብዙሃን አባላት!\n\nበደምቢዶሎ የታገቱ ኢትዮጵያውያን ድምፅ ለ...
5,5,1213760822964609024,positive,የምወድሽ፣\nየምሳሳልሽ፣\nእናቴ፣\nሃገሬ፣ \nኢትዮጵያ!\nከላይ አረንጓ...
6,6,1201134046862094336,positive,rt : የዚህ ቤት ከፍለን የማንጨርሰው ውለታ አለብን \nthe best p...
7,7,1214129980306808834,neutral,ልሙጦቹ ... ደግሞ ምን ይደረግላችሁ በበአሉ?
...,...,...,...,...
931,931,1216256678808735744,neutral,• ከታች ያለው አማራ ክልል ነው! እዚያ አዲሥ ናቸው መሠለኝ ብሩን እያተ...
933,933,1214922390469324802,neutral,_zerihun የውስጥ ሰለማችን መጠበቅ፣ የልማታችን ሁሉ ወሳኝ ማዕከል ነ...
936,936,1223053757883260928,positive,"""አዲሱ መንግስት""ና ህገ-መንግስታዊነት - ረዥም ገመድ\n\nህገ-መንግስታ..."
937,937,1213771045070659584,negative,_astegoma _nega ትግሬ ማለትህ ነው በምቀኝነት የገማቹህ ህዝቦች...
